In [1]:
!pip install imagesize opencv-python 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 45.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install torch==2.1.2+cu118 torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 420.3 kB/s eta 0:00:000:0100:02m
  Obtaining dependency information for torchvision from https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp310-cp310-linux_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 96.6 MB/s eta 0:00:00ta 0:00:01
  Obtaining dependency information for filelock from https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 29.2 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for fsspec from https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.5 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for pillo

In [7]:
!pip install tqdm transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 49.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.4 MB/s eta 0:00:00
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 78.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import json
import imagesize
import torch
import glob
import random
import cv2
import numpy as np
from torch.utils.data import Dataset, RandomSampler, DataLoader, Sampler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

In [2]:
class DLoader(Dataset):
    def __init__(self, sortocr=0, max_seq=1024, tokenizer=None, testmode=0):
        self.dyn_max_seq_len = max_seq
        self.tokenizer = tokenizer
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.testmode = testmode
        self.sortocr = sortocr
        self.data = self.load_cigroupdata()
        print('self.data',len(self.data))

    def __len__(self):
        return len(self.data)
        #return 1
        
    def load_cigroupdata(self):
        samples = 0
        dirp = '/mnt/efs/RaghavWork/CIGroups'
        outsp = {}
        for root, dir, files in os.walk(dirp):
            for file in files:
                if file.endswith('.json'):continue
                key = os.path.basename(root)
                fullpath = os.path.join(root, file)
                fullpath_v2 = os.path.join(root, file+'_geo.json')
                if all(map(lambda x:os.path.isfile(x), [fullpath, fullpath_v2])):
                    if key not in outsp:outsp[key] = []
                    #outsp[key].append((fullpath, fullpath_v2))
                    outsp[key].append((fullpath, fullpath_v2))
                    samples +=1
                    
        allkys = list(outsp.keys())
        count = 0
        datalist = []
        for k in allkys:
            if len(outsp[k])<2:
                allkys.pop(k)
            else:
                count += len(outsp[k])
                datalist.append(outsp.pop(k))
                
        print('Total Samples:',samples, ' | datalist',len(datalist))
        if self.testmode:
            return datalist[-20:]
        else:
            return datalist[:-20]

    def sort_ocr_by_position(self, ocr_data):
        """
        Sort OCR data by reading order (left to right, top to bottom).
        
        Args:
        ocr_data (list): List of tuples in the format [((x1, y1, x3, y3), text)], where 
                         (x1, y1) is the top-left corner and (x3, y3) is the bottom-right corner of the word.
        
        Returns:
        list: Sorted list of tuples by reading order.
        """
        # Sort first by the y1 (top-to-bottom), then by x1 (left-to-right)
        # Example OCR data
        # ocr_data = [
        #     ((100, 50, 150, 80), 'hello'),
        #     ((200, 50, 250, 80), 'world'),
        #     ((100, 100, 150, 130), 'foo'),
        #     ((200, 100, 250, 130), 'bar')
        # ]
        
        # # Sort OCR data
        # sorted_data = sort_ocr_by_position(ocr_data)
        
        # Display the sorted OCR data
        # for box, text in sorted_data:
        #     print(f"Text: {text}, Coordinates: {box}")
        
        sorted_ocr = sorted(ocr_data, key=lambda item: (item[0][1], item[0][0]))
    
        combined_text = " ".join([item[1] for item in sorted_ocr])
        
        return sorted_ocr, combined_text
    
    def get_image_and_jsongt(self, sample):
        words = []
        word_boxes = []
        word_labels = []
        row_labels = []
        img = None
        
        imgp, jsonp =  sample
        # if self.loadimage:
        #     img = cv2.imread(imgp)
        # else:img = None
        img = None
        if isinstance(jsonp, str) and os.path.isfile(jsonp):
            with open(jsonp, encoding="utf8") as f:
                jsondata = json.load(f)
        elif isinstance(jsonp, dict):jsondata = jsonp
            
    
        for clas in jsondata['parse']['class']:
            items = jsondata['parse']['class'][clas]
            for item in items:
                for wrd_id in item:
                    word = jsondata['words'][wrd_id]['text']
                    #print(f'Word: {word} --> {clas}')
                    #words.append(word)
                    #word_boxes.append(jsondata['words'][wrd_id]['boundingBox'][0] + jsondata['words'][wrd_id]['boundingBox'][2])
                    #word_boxes.append(((jsondata['words'][wrd_id]['boundingBox'][0] + jsondata['words'][wrd_id]['boundingBox'][2]), word))
                    #word_labels.append(clas)
                    rlabel = jsondata['words'][wrd_id].get('row_label', [0])[0]
                    #row_labels.append(min(49, rlabel))
                    word_boxes.append(((jsondata['words'][wrd_id]['boundingBox'][0] + jsondata['words'][wrd_id]['boundingBox'][2]),\
                                       word, clas, rlabel))
                    
        #print('words', len(words),' | word_boxes:',len(word_boxes), ' | word_labels:',len(word_labels))
    
        return imgp, img ,words, word_boxes, word_labels, row_labels

    def get_prompt(self, ocr_formatted_1, ocr_formatted_2, example_output):
        # prompt = "Extract the line-item json from the ocr information of the document."
        prompt = 'Your task is to extract line-items information from the attached invoice. Refer to the example \
        input and output and then return the output for the new input.'
        
        one_shot_prompt = f"""
            Task: {prompt}
            
            Example:
            Input: {ocr_formatted_1}
            Output: {example_output}
            
            New Input: {ocr_formatted_2}
            Output:
            """
        return one_shot_prompt
    
    def get_final_json(self, sample):
        row_wise_data = {'keyvalues':{}, 'lineitems':{}}
        kvf = 1
        for item in sample:
            bx, word, cls, row = item
            if row == 0:
                if cls in ('PO_NUMBER_VALUE', 'HTS_NUMBER_VALUE', 'INVOICE_NUMBER_VALUE'):
                    #if cls not in row_wise_data['keyvalues']:row_wise_data['keyvalues'][cls] = []
                    #row_wise_data['keyvalues'][cls].append(word)
                    if cls not in row_wise_data['keyvalues']:row_wise_data['keyvalues'][cls] = ''
                    row_wise_data['keyvalues'][cls] += word + ' '
                    kvf = 0
                continue
            if not cls.endswith('_VALUE') or 'MISC' in cls:continue
            if row not in row_wise_data['lineitems']:row_wise_data['lineitems'][row] = {}
            # if cls not in row_wise_data['lineitems'][row]:row_wise_data['lineitems'][row][cls] = []
            # row_wise_data['lineitems'][row][cls].append(word)
            if cls not in row_wise_data['lineitems'][row]:row_wise_data['lineitems'][row][cls] = ''
            row_wise_data['lineitems'][row][cls] += word + ' '
        
        if kvf:row_wise_data.pop('keyvalues')
        for row in row_wise_data['lineitems']:
            for cls in row_wise_data['lineitems'][row]:
                row_wise_data['lineitems'][row][cls] = row_wise_data['lineitems'][row][cls].strip()
                
        return row_wise_data
    
    def __getitem__(self, idx, testing_sample=0):
        #idx = np.random.randint(0,len(self.data)))
        sample1, sample2 = random.choices(self.data[idx], k = 2)
        #sample1, sample2 = self.data[idx][0], self.data[idx][1]
        #print('sample1, sample2',sample1, sample2)
        imgp1, img1 ,words1, word_boxes1, word_labels1, row_labels1 = self.get_image_and_jsongt(sample1)
        imgp2, img2 ,words2, word_boxes2, word_labels2, row_labels2 = self.get_image_and_jsongt(sample2)
        
        if self.sortocr:
            word_boxes1, ocrtxt1 = self.sort_ocr_by_position(word_boxes1)
            word_boxes2, ocrtxt2 = self.sort_ocr_by_position(word_boxes2)
        else:
            ocrtxt1 = ' '.join([item[1] for item in word_boxes1])
            ocrtxt2 = ' '.join([item[1] for item in word_boxes2])

        outjson1 = self.get_final_json(word_boxes1)
        outjson2 = self.get_final_json(word_boxes2)
        input_prompt = self.get_prompt(ocrtxt1, ocrtxt2, outjson1)
        output = f"{outjson2}"
        if not testing_sample:
            #print('input_prompt--->',input_prompt)
            #print('output--->',output)
            combined_text = input_prompt + output
            tokens = self.tokenizer(
                combined_text, 
                max_length=self.dyn_max_seq_len, 
                truncation=True, 
                return_tensors="pt", 
                padding="max_length",
                padding_side='right'
            )
            input_length = len(self.tokenizer(input_prompt)["input_ids"])
            labels = tokens["input_ids"].clone()
            labels[:, :input_length] = -100  # Ignore input tokens in the loss
            labels[tokens["attention_mask"]==0] = -100 # Ignore pad tokens in the loss
            
            #return {'input_prompt':input_prompt, 'output':output}
            return {
                "input_ids": tokens["input_ids"].squeeze(0),
                "attention_mask": tokens["attention_mask"].squeeze(0),
                "labels": labels.squeeze(0)
            }
        else:
            return {'input_prompt':input_prompt, 'output':output, 'imgp1':imgp1, 'imgp2':imgp2}

In [28]:
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
## tokenizer

local_cache = './deepseek_model/'
name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# local_cache = './mistral_models'
# name = "mistralai/Mistral-7B-Instruct-v0.3"

#http://localhost:8889/edit/mnt/efs/RaghavWork/VirtualENV/1shot/lib/python3.10/site-packages/transformers/models/mistral/modeling_mistral.py
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(name, cache_dir=local_cache)
tokenizer.pad_token = tokenizer.eos_token  # For causal LM

In [5]:
# dataset

fact = 3
max_inp_seq = 1024*fact
train_dataset = DLoader(sortocr=1, max_seq=max_inp_seq, tokenizer=tokenizer, testmode=0)
#train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

val_dataset = DLoader(sortocr=1, max_seq=max_inp_seq, tokenizer=tokenizer, testmode=0)
#val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)
print('MaxLeN:',max_inp_seq)

Total Samples: 660  | datalist 169
self.data 149
Total Samples: 660  | datalist 169
self.data 149
MaxLeN: 3072


In [16]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype="float16",  # Use fp16 for computation
    bnb_4bit_use_double_quant=True,   # Use double quantization
    bnb_4bit_quant_type="nf4",        # Use `nf4` quantization type
)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    name,
    cache_dir=local_cache,
    quantization_config=bnb_config,
    device_map="auto"  # Automatically distribute model across GPUs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Model
# model = AutoModelForCausalLM.from_pretrained(
#     name,
#     cache_dir=local_cache,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# Set to evaluation mode and freeze original weights
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Data collator

"""
A DataCollator in Hugging Face is used by the Trainer to collate multiple individual samples into a batch. When those samples have variable lengths, the DataCollator:

  1.  Pads them dynamically (if needed)
  2.  Optionally applies masking (for MLM)
  3.  Ensures all samples in a batch have same shape
"""

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked LM for causal LM
)

In [9]:
from peft import LoraConfig, get_peft_model, TaskType

# Configure LoRA
lora_config = LoraConfig(
    r=8,             # Rank of the low-rank matrices
    lora_alpha=16,   # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Target attention layers to apply LoRA
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM,  # Specify the task
    bias="none"
)

# Add LoRA to the model
model = get_peft_model(model, lora_config)

In [11]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./deepseek_output",         # Directory to save results
    eval_strategy="epoch",         # Evaluate at the end of each epoch
    max_grad_norm=1.0,                   # Clip gradients
    learning_rate=5e-5,                  # Learning rate
    per_device_train_batch_size=1,       # Batch size per GPU
    per_device_eval_batch_size=1,        # Evaluation batch size per GPU
    num_train_epochs=25,                  # Number of epochs
    weight_decay=0.01,                   # Weight decay for optimizer
    save_strategy="epoch",               # Save model checkpoint at the end of each epoch
    save_total_limit=2,                  # Keep only the last 2 checkpoints
    load_best_model_at_end=True,         # Load the best model when finished
    logging_dir="./logs",                # Directory for logs
    logging_steps=50,                    # Log every 50 steps
    report_to="tensorboard",             # Use TensorBoard for logging
    fp16=True,                           # Use mixed precision (16-bit floating point)
    dataloader_num_workers=0,            # Number of data loader workers
    ddp_find_unused_parameters=True,     # Optimize for distributed training
)



# training_args = TrainingArguments(
#     output_dir="./deepseek_output",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     gradient_accumulation_steps=4,         # simulate larger batch size
#     num_train_epochs=20,                    # you can increase if needed
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     max_grad_norm=1.0,
#     # save_total_limit=2,
#     load_best_model_at_end=True,
#     logging_dir="./logs",
#     logging_steps=20,
#     fp16=True,
#     report_to="tensorboard",               # or "none"
#     dataloader_num_workers=0,              # increase if not using Jupyter
#     # ddp_find_unused_parameters=True        # if using multi-GPU
# )

In [12]:
# Use Trainer API for training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [15]:
torch.cuda.empty_cache()

In [16]:
# Train the model
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 2 has a total capacty of 22.19 GiB of which 99.50 MiB is free. Including non-PyTorch memory, this process has 22.09 GiB memory in use. Of the allocated memory 21.50 GiB is allocated by PyTorch, and 288.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
!pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 7.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install -U bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [13]:
!pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 59.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 127.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 124.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.2/316.2 kB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 34.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [18]:
class MultiDeviceTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Move inputs to the correct device of model's embeddings
        device = model.get_input_embeddings().weight.device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Compute loss
        outputs = model(**inputs)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

In [7]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
    # data_collator=data_collator
)

/tmp/ipykernel_174746/3608974816.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train
trainer.train()

In [14]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [1]:
print("hello")

hello
